<img src="https://davewritescode.com/wp-content/uploads/2019/10/20170815-rock-scissors-pixabay-34f3h8kx0ucail5dbcp8n4-1024x607.jpg" width=600>

Computer Vision with CNNs

Build a classifier for Rock-Paper-Scissors based on the rock_paper_scissors TensorFlow dataset.


In [ ]:


import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

# Loading Dataset from Kaggle To Try for Myself

In [ ]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D, AveragePooling2D, Input, BatchNormalization
import pandas as pd
import seaborn as sns
from tensorflow.keras.regularizers import l2
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt

In [ ]:
api_token = {"username":"bradleybolus","key":"f9ded31baecf6b5012ecf8faacbeaa0d"}

In [ ]:
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sanikamal/rock-paper-scissors-dataset

 98% 445M/452M [00:04<00:00, 85.7MB/s]
100% 452M/452M [00:04<00:00, 102MB/s] 


# Unzipping Image Data from Rock Paper Scissors Dataset

In [ ]:
import zipfile
import os

In [ ]:
train_zip_path = "/content/rock-paper-scissors-dataset.zip"

zip_ref = zipfile.ZipFile(train_zip_path, 'r')

zip_ref.extractall('.')
zip_ref.close()

In [ ]:
base_dir = '/content/Rock-Paper-Scissors'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
print(train_dir)
print(val_dir)
print(test_dir)

/content/Rock-Paper-Scissors/train
/content/Rock-Paper-Scissors/val
/content/Rock-Paper-Scissors/test


In [ ]:
len(train_dir)


34

In [ ]:
len(val_dir)


32

In [ ]:
len(test_dir)

33

# Model 1: Baseline Model

In [ ]:
model_base = tf.keras.models.Sequential([

Conv2D(32,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'),
MaxPooling2D(pool_size =(2,2),strides=2),
Conv2D(32,kernel_size=(3,3),activation='relu'),
MaxPooling2D(pool_size =(2,2),strides=2),
Conv2D(32,kernel_size=(3,3),activation='relu'),
MaxPooling2D(pool_size =(2,2),strides=2),
Conv2D(32,kernel_size=(3,3),activation='relu'),
MaxPooling2D(pool_size =(2,2),strides=2),
Conv2D(32,kernel_size=(3,3),activation='relu'),
MaxPooling2D(pool_size =(2,2),strides=2),
Flatten(),

Dense(units=256,activation='relu'),
Dense(units=256,activation='relu'),
Dense(units=256,activation='relu'),
Dense(units=128,activation='relu'),
Dense(units=64,activation='relu'),
Dense(units=32,activation='relu'),
Dense(units=3,activation='softmax')])

In [ ]:
model_base.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 32)        9

In [ ]:
train_data_dir = "/content/Rock-Paper-Scissors/train"
val_data_dir = "/content/Rock-Paper-Scissors/validation"
test_data_dir = "/content/Rock-Paper-Scissors/test"

train_datagen = ImageDataGenerator(
    rescale= 1./ 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode = "nearest")
# Try 0.1
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [ ]:
model_base.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath="brad_best_rock.h5",
                                                 save_weights_only=False, # the whole model (False) or only weights (True) 
                                                 save_best_only=True, # keep the best model with lowest validation loss
                                                 monitor='val_loss',
                                                 verbose=1)

In [ ]:
history_base = model_base.fit(train_generator, validation_data=test_generator, callbacks = checkpoint_callback,
                    epochs=6,
                    verbose=2)

Epoch 1/6
79/79 - 23s - loss: 0.2073 - accuracy: 0.9266 - val_loss: 0.0771 - val_accuracy: 0.9624

Epoch 00001: val_loss improved from inf to 0.07711, saving model to brad_best_rock.h5
Epoch 2/6
79/79 - 21s - loss: 0.1810 - accuracy: 0.9294 - val_loss: 0.1856 - val_accuracy: 0.9409

Epoch 00002: val_loss did not improve from 0.07711
Epoch 3/6
79/79 - 21s - loss: 0.1269 - accuracy: 0.9552 - val_loss: 0.1752 - val_accuracy: 0.9462

Epoch 00003: val_loss did not improve from 0.07711
Epoch 4/6
79/79 - 21s - loss: 0.1565 - accuracy: 0.9464 - val_loss: 0.0668 - val_accuracy: 0.9677

Epoch 00004: val_loss improved from 0.07711 to 0.06684, saving model to brad_best_rock.h5
Epoch 5/6
79/79 - 21s - loss: 0.1355 - accuracy: 0.9504 - val_loss: 0.1839 - val_accuracy: 0.9382

Epoch 00005: val_loss did not improve from 0.06684
Epoch 6/6
79/79 - 21s - loss: 0.1252 - accuracy: 0.9548 - val_loss: 0.1104 - val_accuracy: 0.9543

Epoch 00006: val_loss did not improve from 0.06684


In [ ]:
model_base.evaluate(test_generator)

12/12 [==============================] - 1s 105ms/step - loss: 0.1104 - accuracy: 0.9543


[0.11036036163568497, 0.9543010592460632]

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
restored_model = tf.keras.models.load_model('brad_best_rock.h5')

# Show the model architecture
restored_model.summary()

# We need to define its optimizer and loss function again since the h5 file
# does not contain those information :(
restored_model.compile(optimizer="adam",
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

# Re-evaluate the model
loss, acc = restored_model.evaluate(test_generator)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 32)        9

In [ ]:
def solution_model():
  url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
  urllib.request.urlretrieve(url, 'rps.zip')
  local_zip = 'rps.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('tmp/')
  zip_ref.close()

  ##################################
  
  TRAINING_DIR = "tmp/rps/"
  
  train_datagen = ImageDataGenerator(
  rescale= 1./ 255,
  shear_range=0.2,
  zoom_range=0.2,
  rotation_range = 40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True,
  fill_mode = "nearest")

  test_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(
      TRAINING_DIR,
      target_size=(150, 150),
      batch_size=32,
      class_mode='categorical')
  
  #######################################
  
  model = tf.keras.models.Sequential([

  Conv2D(32,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'),
  MaxPooling2D(pool_size =(2,2),strides=2),
  Conv2D(32,kernel_size=(3,3),activation='relu'),
  MaxPooling2D(pool_size =(2,2),strides=2),
  Conv2D(32,kernel_size=(3,3),activation='relu'),
  MaxPooling2D(pool_size =(2,2),strides=2),
  Conv2D(32,kernel_size=(3,3),activation='relu'),
  MaxPooling2D(pool_size =(2,2),strides=2),
  Conv2D(32,kernel_size=(3,3),activation='relu'),
  MaxPooling2D(pool_size =(2,2),strides=2),
  Flatten(),

  Dense(units=256,activation='relu'),
  Dense(units=256,activation='relu'),
  Dense(units=256,activation='relu'),
  Dense(units=128,activation='relu'),
  Dense(units=64,activation='relu'),
  Dense(units=32,activation='relu'),
  Dense(units=3,activation='softmax')])

  #######################################
  
  model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ['accuracy'])
  
  #######################################

  model.fit(train_generator,epochs=6,verbose=2)

  return model

if __name__ == '__main__':
  model = solution_model()
  model.save("brad_rock_1.h5")

Found 2520 images belonging to 3 classes.
Epoch 1/6
79/79 - 21s - loss: 1.0639 - accuracy: 0.3913
Epoch 2/6
79/79 - 20s - loss: 0.8811 - accuracy: 0.5563
Epoch 3/6
79/79 - 21s - loss: 0.6441 - accuracy: 0.7063
Epoch 4/6
79/79 - 21s - loss: 0.5324 - accuracy: 0.7591
Epoch 5/6
79/79 - 20s - loss: 0.4617 - accuracy: 0.8099
Epoch 6/6
79/79 - 20s - loss: 0.3415 - accuracy: 0.8635


In [ ]:
model.fit(train_generator,epochs=6,verbose=2)

Epoch 1/6
79/79 - 20s - loss: 0.2289 - accuracy: 0.9179
Epoch 2/6
79/79 - 20s - loss: 0.2360 - accuracy: 0.9107
Epoch 3/6
79/79 - 20s - loss: 0.1565 - accuracy: 0.9488
Epoch 4/6
79/79 - 20s - loss: 0.1468 - accuracy: 0.9552
Epoch 5/6
79/79 - 20s - loss: 0.1363 - accuracy: 0.9528
Epoch 6/6
79/79 - 20s - loss: 0.1164 - accuracy: 0.9611


In [ ]:
model.save("brad_rock_2.h5")

# Model 1 Results: 96 %

# Model 2: Xception

In [ ]:
from tensorflow import keras

In [ ]:
def solution_model():
  url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
  urllib.request.urlretrieve(url, 'rps.zip')
  local_zip = 'rps.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('tmp/')
  zip_ref.close()

  ##################################
  
  TRAINING_DIR = "tmp/rps/"
  
  train_datagen = ImageDataGenerator(
  rescale= 1./ 255,
  shear_range=0.2,
  zoom_range=0.2,
  rotation_range = 40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True,
  fill_mode = "nearest")

  test_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(
      TRAINING_DIR,
      target_size=(150, 150),
      batch_size=32,
      class_mode='categorical')
  
  #######################################
  model_X = keras.applications.Xception(
  weights='imagenet',  # Load weights pre-trained on ImageNet.
  input_shape=(150, 150, 3),
  include_top=False)  #


  model_X.trainable = False

  inputs = keras.Input(shape=(150, 150, 3))

  x = model_X(inputs, training=False)

  x = keras.layers.GlobalAveragePooling2D()(x)

  outputs = keras.layers.Dense(3, activation = "softmax")(x)
  model_X = keras.Model(inputs, outputs)

  #######################################
  
  model_X.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ['accuracy'])
  
  #######################################

  model_X.fit(train_generator,epochs=10,verbose=2)

  return model_X

if __name__ == '__main__':
  model_X = solution_model()
  model_X.save("brad_rock_X.h5")

Found 2520 images belonging to 3 classes.
Epoch 1/10
79/79 - 19s - loss: 0.5462 - accuracy: 0.7956
Epoch 2/10
79/79 - 17s - loss: 0.2813 - accuracy: 0.9202
Epoch 3/10
79/79 - 17s - loss: 0.2121 - accuracy: 0.9405
Epoch 4/10
79/79 - 17s - loss: 0.1673 - accuracy: 0.9587
Epoch 5/10
79/79 - 17s - loss: 0.1515 - accuracy: 0.9540
Epoch 6/10
79/79 - 17s - loss: 0.1333 - accuracy: 0.9655
Epoch 7/10
79/79 - 17s - loss: 0.1154 - accuracy: 0.9734
Epoch 8/10
79/79 - 17s - loss: 0.1047 - accuracy: 0.9770
Epoch 9/10
79/79 - 17s - loss: 0.1023 - accuracy: 0.9706
Epoch 10/10
79/79 - 17s - loss: 0.0983 - accuracy: 0.9718


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


# XCeption Results 97 %